In [73]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math

In [74]:
class Value:

    def __init__(self, data, _children=(), _op=''):
        self.grad = 0.0
        self.data = data
        self._prev = set(_children)
        self._op = _op
        self._backward = lambda : None

    def __repr__(self):
        return f"{self.data}"

    def __neg__(self):
      return self * -1

    def __sub__(self, other):
      return self + (-other)

    def __add__(self, other):

        other = other if isinstance(other, Value) else Value(other)
        out = Value((self.data+other.data), (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad

        out._backward = _backward
        return out

    def __radd__(self, other): # other + self
        return self + other

    def __mul__(self, other):

        other = other if isinstance(other, Value) else Value(other)
        out =  Value((self.data*other.data), (self, other), '*')

        def _backward():
            self.grad += out.grad*other.data
            other.grad += out.grad*self.data

        out._backward = _backward

        return out

    def __rmul__(self, other):
        return self*other

    def tanh(self):
        x = self.data
        y = (math.exp(2*x)-1)/(math.exp(2*x)+1)
        out = Value(y , (self,))

        def _backward():
            self.grad += (1-y**2) * out.grad

        out._backward = _backward
        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float))
        out = Value(self.data**other, (self,))

        def _backward():
            self.grad += (other * self.data**(other-1)) * out.grad
        out._backward = _backward

        return out


    def __truediv__(self, other):
        return self * (other**-1)

    def backward(self):
        topo = []
        visited = set()

        def topo_sort(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    topo_sort(child)
                topo.append(v)

        topo_sort(self)

        self.grad = 1

        for node in reversed(topo):
            node._backward()


In [75]:
class Neuron:

    def __init__(self, nin):

        self.w = [ Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))

    def __call__(self, x):

        act = sum((wi*xi for wi, xi in zip(self.w,x)), self.b)
        out = act.tanh()
        return out

    def parameters(self):
      return self.w + [self.b]

class Layer:

    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self,x):
        out = [n(x) for n in  self.neurons]
        return out[0] if len(out) == 1 else out

    def parameters(self):
      return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:

    def __init__(self, nin, nout):

        sz = [nin]+nout
        self.layers = [Layer(sz[x], sz[x+1]) for x in range(len(nout))]

    def __call__(self,x):

        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
      return [p for layer in self.layers for p in layer.parameters()]




In [76]:
x = [3.0, -2.0]
n = MLP(2,[3,3,1])

n(x)

-0.9107449213394302

In [77]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] # desired targets

In [88]:
for k in range(100):

  # forward pass
  ypred = [n(x) for x in xs]
  loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))

  # backward pass
  for p in n.parameters():
    p.grad = 0.0
  loss.backward()

  # update
  for p in n.parameters():
    p.data += -0.01 * p.grad

print(k, loss.data)

99 0.05613682134510456


In [89]:
ypred

[0.9939994939312133,
 -0.9401894190444228,
 -0.8454237176354058,
 0.8307969190347865]